In [2]:
import requests
from bs4 import BeautifulSoup
import csv
import pandas as pd
import string
from collections import Counter
import numpy as np

## To have in the data collection :

In [ ]:
def url_to_label(url):
    last_part = url.rstrip('/').split('/')[-1]  # We only keep the last part of the url
    label = ' '.join([word.capitalize() for word in last_part.replace('-', ' ').replace('_', ' ').split()])  # We delete unwanted element ("_") and replace them by a space. The following letter is set as a capital letter.
    return label

In [ ]:
import csv

def create_nodes_csv(all_links, output_file="nodes.csv", label_function=None):
    nodes = set(all_links.keys())  # Each key of the dictionnary "all_links" is a node (we don't keep any possible duplicate)
    for targets in all_links.values():
        nodes.update(targets)  # Each value of the dictionnary is a node

    with open(output_file, mode="w", newline="", encoding="utf-8") as f:  # Create (or open) a csv file 
        writer = csv.writer(f)  # Create an object that will enable our code to write within the file

        if label_function:
            writer.writerow(["node_id", "label"])  # If we want different labels than the url, we can give the function an other function that will creates those labels
        else:
            writer.writerow(["node_id"])  # If no function to labelize was given, we just have the node_id in the file
        print("This function is used to labelize the urls:", label_function)
        for node in nodes:
            if label_function:
                try:
                    label = label_function(node)  # We write in the column "label" the label given by the function
                except Exception:
                    label = node  # In case of error, we keep the url (the node) as the label
                writer.writerow([node, label])
            else:
                writer.writerow([node])  # If there is no label, we just write the url of the node in the write column

    print(f"Nodes CSV created: {len(nodes)} nodes")

In [ ]:
def create_edges_csv(links, output_file="edges.csv"):
    
    with open(output_file, mode="w", newline="", encoding="utf-8") as f:
        writer = csv.writer(f)
        writer.writerow(["source", "target"])  # We start by writing the headers of our column

        for source, targets in links.items():
            for target in targets:  # Mettre "set(targets)"" si je veux éviter les doublons (A -> B), il comprend bien que A -> B x=x B -> A donc tkt
                # A ajouter / tester (devrait empecher les boucles infines)
                if target != source:  
                    writer.writerow([source, target])  # If the target is different form the source (not A -> A), we add the relation in our file
                else :
                    continue

    print(f"Edges CSV created: {edge_count} edges")

# ---------------------------------------------------------------------------------------------------------------------------------------------------------

IMPORTANT : pour le code ajouter une ligne pour que l'élément html brut csv file soit non pas l'élément obtenu de la fonction mais le fichier télécharger dans la machine (plus simple pour faire fonctionner le code sans refaire la fonction)  -> pour l'extraction et link analysis

import pandas as pd

wiki_nodes = pd.read_csv("data/wikipedia_nodes.csv")
wiki_edges = pd.read_csv("data/wikipedia_edges.csv")

devrait fonctionctionner ??

In [3]:
wiki_nodes_path = r"C:\Users\Violaine\OneDrive\Ecole\Master 1\Web mining\Projet Web Mining\Projet_Web_Mining\wikipedia_nodes.csv"
wiki_edges_path = r"C:\Users\Violaine\OneDrive\Ecole\Master 1\Web mining\Projet Web Mining\Projet_Web_Mining\wikipedia_edges.csv"

wiki_nodes = pd.read_csv(wiki_nodes_path)
wiki_edges = pd.read_csv(wiki_edges_path)

print(len(wiki_edges))
print(len(wiki_nodes))

1701
1639


In [4]:
# Useful to check if all nodes present in the edges file are in the nodes file as well (the result here should be 0)
invalid_edges = wiki_edges[
    ~wiki_edges['source'].isin(wiki_nodes['id']) |
    ~wiki_edges['target'].isin(wiki_nodes['id'])
]

len(invalid_edges)

0

## Computation of the Adjacency Matrix

In [12]:
import pandas as pd
import numpy as np

wiki_node_ids = wiki_nodes['id'].tolist()  # Transform the nodes ids into a list

wiki_node_to_idx = {node_id: idx for idx, node_id in enumerate(wiki_node_ids)}  # Create a dictionnary where each node id correspond to a number (key : id)
wiki_idx_to_node = {idx: node_id for node_id, idx in wiki_node_to_idx.items()}  # Create a dictionnary where each node id correspond to a number (key : number)

In [7]:
n = len(wiki_node_ids)  # Count the number of nodes (the adjancy matrix will have a n x n format)
wiki_adjacency_matrix = np.zeros((n, n), dtype=int)  # Initialising the matrix with 0 with the correct number of rows and columns (= n x n)

for _, row in wiki_edges.iterrows():  # For each row of the wiki edges document
    source = row['source']
    target = row['target']

    if source in wiki_node_to_idx and target in wiki_node_to_idx:  # Verify that each source and target nodes of the edge document is in the nodes document
        i = wiki_node_to_idx[source]
        j = wiki_node_to_idx[target]

        wiki_adjacency_matrix[i, j] += 1
        #wiki_adjacency_matrix[j, i] += 1  --> It's an oriented graph so we don't increment the nodes who goes : target -> source

In [8]:
wiki_adjacency_df = pd.DataFrame( wiki_adjacency_matrix, index = wiki_node_ids, columns = wiki_node_ids)

print(wiki_adjacency_matrix)
print(wiki_adjacency_df)

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
                                                    https://en.wikipedia.org/wiki/Douglas_Kellner  \
https://en.wikipedia.org/wiki/Douglas_Kellner                                                   0   
https://en.wikipedia.org/wiki/Cliff_jumping                                                     0   
https://en.wikipedia.org/wiki/Walkway                                                           0   
https://en.wikipedia.org/wiki/Cicero                                                            0   
https://en.wikipedia.org/wiki/The_Antonio_Treat...                                              0   
...                                                                                           ...   
https://en.wikipedia.org/wiki/Khedive                                                           0   
https://en.wikipedia.org/wiki/Straight_leg_jeans                        

In [ ]:
# The result should be = to the number of edges (if graph oriented)
np.count_nonzero(wiki_adjacency_matrix)

np.int64(1701)

## Shortest Path

In [ ]:
import numpy as np
import pandas as pd
from collections import deque

def shortest_path(adjacency_matrix, start_node, end_node, node_to_idx, idx_to_node):
    n = adjacency_matrix.shape[0]  # = len(wiki_node_ids) 
    visited = [False] * n  # List of visited nodes set to 0 nodes visited and with a lenght of n
    prev = [None] * n  # Stock the previous nodes visited (useful to construct the final path) and with a lenght of n
    
    queue = deque()  # Set a queue
    start_idx = node_to_idx[start_node]  # Starting point of our path transform from a link to its adjacency matrix's index
    end_idx = node_to_idx[end_node]  # Ending point of our path transform from a link to its adjacency matrix's index
    
    queue.append(start_idx)  # We set the starting point in the queue 
    visited[start_idx] = True  # Set the starting node as visited
    
    while queue:
        current = queue.popleft()  # While the queue is not empty, we explore the oldest element of the queue (-> popleft)
        
        if current == end_idx:  # If the current node is the end node, than we create the path
            path = []
            while current is not None:
                path.append(idx_to_node[current])
                current = prev[current]  # We create the path from the end node and add to it the nodes stock in the prev list
            return path[::-1]  # We replace the nodes of the path in the correct order and return the shortest path
        
        neighbors = np.where(adjacency_matrix[current] > 0)[0]  # Return all the indexe of the neighbors of the current node where the relation current -> neigbhor is not nul
        for neighbor in neighbors:
            if not visited[neighbor]:  
                visited[neighbor] = True  # If the neighbor is not already visited, we add it to the visited list
                prev[neighbor] = current  # We set the current node as the previous one to the neighbor
                queue.append(neighbor)  # And add the neighbor to the queue to explore it later
    
    return None  # If their is no path

In [ ]:
start_node = 'https://en.wikipedia.org/wiki/Lifestyle_trends_and_media'  # Remplace par l'id réel du nœud de départ
end_node = 'https://en.wikipedia.org/wiki/Celebrity'    # Remplace par l'id réel du nœud de destination

path = shortest_path( wiki_adjacency_matrix, start_node, end_node, wiki_node_to_idx, wiki_idx_to_node)

if path:
    print(f"The shortest path between {start_node} and {end_node} is :")
    print(" -> ".join(path))
    print("And has a lenght (/ diameter) of", len(path))
else:
    print(f"No path between {start_node} and {end_node} was found")


The shortest path between https://en.wikipedia.org/wiki/Lifestyle_trends_and_media and https://en.wikipedia.org/wiki/Celebrity is :
https://en.wikipedia.org/wiki/Lifestyle_trends_and_media -> https://en.wikipedia.org/wiki/Celebrity
And has a lenght (/ diameter) of 2


## Degree Matrix

In [ ]:
def degree_matrix(A: np.ndarray, direction: str = "out") -> np.ndarray:
    if direction == "out":
        deg = A.sum(axis = 1)  # The out-degree is the number of edges that leave a node. Inn the adjacency matrix A, rows represent source nodes and columns represent target nodes. Summing each row (axis=1) gives the total number of outgoing edges for each node.
    elif direction == "in":
        deg = A.sum(axis = 0)  # Same principle but the opposite
    return np.diag(deg)

wiki_degree_matrix = degree_matrix(wiki_adjacency_df)
print(wiki_degree_matrix)

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


In [28]:
np.count_nonzero(wiki_degree_matrix)

np.int64(13)

### Degree centrality

In [10]:
import numpy as np
import pandas as pd

def degree_centrality(adj_matrix: np.ndarray, node_ids: list) -> pd.DataFrame:
    out_deg_vec = adj_matrix.sum(axis=1)  # Compute the out degree of the adjacency matrix (= sum on rows)
    in_deg_vec  = adj_matrix.sum(axis=0)  # Compute the in degree of the adjacency matrix (= sum on columns)
    total_deg_vec = out_deg_vec + in_deg_vec  # Compute the total degree
    
    degree_df = pd.DataFrame({ "node_id": node_ids, "out_degree": out_deg_vec, "in_degree": in_deg_vec, "total_degree": total_deg_vec})
    
    degree_df = degree_df.sort_values(by="total_degree", ascending=False).reset_index(drop=True)  # Sort the total degrees (highest to lowest)
    
    return degree_df

In [ ]:
degree_df = degree_centrality(wiki_adjacency_matrix, wiki_node_ids)

# Print only the 10 most central nodes
print(degree_df.head(10))

# Optionnel : sauvegarder dans un CSV
#degree_df.to_csv("degree_centrality.csv", index=False)


                                             node_id  out_degree  in_degree  \
0   https://en.wikipedia.org/wiki/Sustainable_living         289          0   
1              https://en.wikipedia.org/wiki/Fashion         276          0   
2      https://en.wikipedia.org/wiki/Interior_design         218          0   
3   https://en.wikipedia.org/wiki/Outdoor_recreation         191          0   
4        https://en.wikipedia.org/wiki/Personal_care         141          0   
5      https://en.wikipedia.org/wiki/Quality_of_life         126          1   
6         https://en.wikipedia.org/wiki/Healthy_diet         105          0   
7               https://en.wikipedia.org/wiki/Travel          81          0   
8            https://en.wikipedia.org/wiki/Self-care          70          0   
9  https://en.wikipedia.org/wiki/Lifestyle_trends...          55          0   

   total_degree  
0           289  
1           276  
2           218  
3           191  
4           141  
5           127  
6   

## Page Rank

In [14]:
def transition_matrix(A: np.ndarray, direction: str = "out") -> np.ndarray:
    if direction == "out":  # If probabilities are based on outgoing ("out")
        deg = A.sum(axis=1)  # Compute the out-degree of each node by summing rows 
        P = np.zeros_like(A, dtype=float)  # Initialize the transition matrix with zeros (P has the same size as A)
        nonzero = deg > 0  # Identify nodes with at least one outgoing edge (to avoid division by zero)
        P[nonzero] = A[nonzero] / deg[nonzero, None]  # Normalize each row: divide outgoing edges by the node's out-degree
        return P

    elif direction == "in":  # If probabilities are based on incoming ("in") edges
        deg = A.sum(axis=0)   
        P = np.zeros_like(A, dtype=float)
        nonzero = deg > 0
        P[:, nonzero] = A[:, nonzero] / deg[nonzero]
        return P

    else:
        raise ValueError("direction must be 'out' or 'in'.")

In [ ]:
def pagerank_eigen(A: np.ndarray, alpha: float = 0.85) -> np.ndarray:  # Compute the PageRank vector using the eigenvector method.
    n = A.shape[0]  # Get the number of nodes in the graph (= size of the adjacency matrix)
    P = transition_matrix(A, direction = "out")  # Build the transition probability matrix based on the outgoing edges from the adjacency matrix
    E = np.ones((n,n)) / n  # Create the teleportation matrix with uniform probabilities (1/n)
    G = alpha * P + (1 - alpha) * E  # Build the Google matrix as a combination of the transition and teleportation matrices

    values, vectors = np.linalg.eig(G.T)  # Compute eigenvalues and eigenvectors of the transpose of the Google matrix

    idx = np.argmax(values)  # Find the index of the largest eigenvalue
    pr = np.real(vectors[:, idx])  # Extract the dominant eigenvector and keep its real part

    pr = pr / pr.sum()  # Normalize the PageRank vector so that its values sum to 1
    return pr

In [19]:
# Compute the PageRank scores for all nodes in the adjacency graph using the eigenvector method
wiki_pr_scores = pagerank_eigen(wiki_adjacency_matrix, alpha=0.85)

# Store the PageRank scores in a DataFrame alongside the corresponding node IDs
wiki_pr_df = pd.DataFrame({ "node_id": wiki_node_ids, "pagerank": wiki_pr_scores})

# Sort the DataFrame by PageRank in descending order to rank nodes from most to least important
wiki_pr_df = wiki_pr_df.sort_values(by="pagerank", ascending=False).reset_index(drop=True)

# Print the 10 most important nodes (highest PageRank scores)
print(wiki_pr_df.head(10))

# Optionnel : sauvegarder dans un CSV
#wiki_pr_df.to_csv("pagerank_scores.csv", index=False)

                                             node_id  pagerank
0               https://en.wikipedia.org/wiki/Health  0.000785
1        https://en.wikipedia.org/wiki/Mental_health  0.000774
2    https://en.wikipedia.org/wiki/Depression_(mood)  0.000774
3  https://en.wikipedia.org/wiki/Lifestyle_(socio...  0.000758
4              https://en.wikipedia.org/wiki/Anxiety  0.000747
5    https://en.wikipedia.org/wiki/Physical_exercise  0.000730
6            https://en.wikipedia.org/wiki/Nutrition  0.000710
7  https://en.wikipedia.org/wiki/World_Health_Org...  0.000708
8                 https://en.wikipedia.org/wiki/Yoga  0.000701
9  https://en.wikipedia.org/wiki/Physical_attract...  0.000701


In [20]:
# On utilise pas ça : c'est sur base d'une approximation donc c'est bien pour les énormes graph (millions de neuds) mais sinon, c'est moins exact mathématiquement

def pagerank_power_iteration(A: np.ndarray, alpha: float = 0.85, max_iter: int = 100) -> np.ndarray:  # Compute PageRank scores using the iterative (power) method
    n = A.shape[0]
    P = transition_matrix(A, direction = "out")
    E = np.ones((n,n)) / n
    pr = np.ones((n,1)) / n
    G = alpha * P + (1 - alpha) * E

    for _ in range (max_iter):
        pr = G.T @ pr
        pr = pr/pr.sum()

    return pr.flatten()

In [21]:
# Compute the PageRank scores for all nodes in the adjacency graph using the power method
wiki_pr_scores = pagerank_power_iteration(wiki_adjacency_matrix, alpha=0.85)

# Store the PageRank scores in a DataFrame alongside the corresponding node IDs
wiki_pr_df = pd.DataFrame({ "node_id": wiki_node_ids, "pagerank": wiki_pr_scores})

# Sort the DataFrame by PageRank in descending order to rank nodes from most to least important
wiki_pr_df = wiki_pr_df.sort_values(by="pagerank", ascending=False).reset_index(drop=True)

# Print the 10 most important nodes (highest PageRank scores)
print(wiki_pr_df.head(10))

# Optionnel : sauvegarder dans un CSV
#wiki_pr_df.to_csv("pagerank_scores.csv", index=False)


                                             node_id  pagerank
0               https://en.wikipedia.org/wiki/Health  0.000785
1        https://en.wikipedia.org/wiki/Mental_health  0.000774
2    https://en.wikipedia.org/wiki/Depression_(mood)  0.000774
3  https://en.wikipedia.org/wiki/Lifestyle_(socio...  0.000758
4              https://en.wikipedia.org/wiki/Anxiety  0.000747
5    https://en.wikipedia.org/wiki/Physical_exercise  0.000730
6            https://en.wikipedia.org/wiki/Nutrition  0.000710
7  https://en.wikipedia.org/wiki/World_Health_Org...  0.000708
8  https://en.wikipedia.org/wiki/Physical_attract...  0.000701
9                 https://en.wikipedia.org/wiki/Yoga  0.000701
